# 2. Check if dynamics have finished or not; and whether there has been a problem

In [3]:
import os 

In [4]:
PDBs_name_starlife = []

for file in os.listdir('/home/imartinv/Escritorio/ventana_a_starlife/CNS_PD_Memprot_dynamics'):
    PDB_na = file
    PDBs_name_starlife.append(PDB_na)

The errors we are contemplating are shown by:
- If the 100ns folder is created but empty. --> It will always be created once we run all dynamics, even if some sbatch fails. This is because the dependency is for any kind of output. So if previous sbatches fail, the 100ns will be created anyway but will be empty as the sbatch_P3 does not have the needed files for running its dynamic. 

- If the error3.log does not contain the number of steps completed --> maybe because of the same reason as above.

In [5]:
# We pay attention to this folder because it is the one resultant of the simulation and it is the one that 
# contains the trajectory of the protein. 


dictio_100ns = {} # Empty dictionary to put those PDBs dynamics that have the 100ns folder.
                  # Although having this 100ns folder, it maybe empty or not completed. So the number of steps 
                  # will have to be reviewed in order to see if it has finished. 


dictio_not_100ns_notrunned = [] # Empty dictionary to put those PDBs dynamics that do not have the 100 ns folder. 
                      # The ones that do not have this 100ns folder maybe because their dynamic has not finished
        
PDBs_with_errors = [] # We also create a list of those PDBs that may have some errors.

PDBS_finished_completed = []

gromos_forcefield_num = 0
charm_forcefield_num = 0
both = 0

both_forcefields_pdb = []
gromos_forcefields_pdb = []
charmm_forcefields_pdb = []

for i in PDBs_name_starlife: # For each PDB folder in starlife

    
    files = os.listdir('/home/imartinv/Escritorio/ventana_a_starlife/CNS_PD_Memprot_dynamics/'+i) # We list the items inside each PDB folder
    
    
    values_dict_100ns = [] # We create a list of the values that we want to include in the dictionary dictio_100ns
    
    
    
    
    if ('100ns') in files: # If the folder 100ns is inside the PDB folder
        
        file_inside_100ns = len(os.listdir('/home/imartinv/Escritorio/ventana_a_starlife/CNS_PD_Memprot_dynamics/'+i+'/100ns')) # Then, we check the number of items inside the 100ns folder.
        
        
        if file_inside_100ns != 0: # If there are items inside the 100ns folder
            
            values_dict_100ns.append('100ns --> full') # We write a message indicating that the folder is not empty

            
            dictio_100ns[i[0:4]] = values_dict_100ns # We add the PDB name (key) and this info to the dictio_100ns as it has 100ns. 
        
        else:
            values_dict_100ns.append('100ns --> empty // Error in previous sbatches') # If the 100ns folder is empty, 
                                                                                                    #then we indicate that there maybe an error with another sbatch that did not generate the folders
                                                                                                    # needed for running the sbatch_P3
            
            dictio_100ns[i[0:4]] = values_dict_100ns # We add the PDB name (key) and this info to the dictionary_100ns
            
            PDBs_with_errors.append(i[0:4])    
        
    else: # If the folder 100ns is not found in the PDB folder, then we add this PDB to the list of PDBs 
        dictio_not_100ns_notrunned.append(i[0:4])
        
        
        
        
        
    if ('error3.log') in files: # If the error3.log file is found, then, from this, we can see the number of steps 
                                # already achieved by our dynamic, if there has been a fatal error or if it has been 
                                #stopped due to time limit. 
        
        error_log_3_lines = [] # To eliminate the '\n' character each time we read a line. List of lines without '\n'.
        
        with open('/home/imartinv/Escritorio/ventana_a_starlife/CNS_PD_Memprot_dynamics/'+i+'/error3.log') as error_log_3:
            
            error_log_3_lines_f = error_log_3.readlines() # We read line by line the error3.log file
            
        for sub in error_log_3_lines_f:
            
            error_log_3_lines.append(sub.replace("\n", "")) # To eliminate the '\n' character each time we read a line.We substitute for nothing.
            


        steps = [] # There maybe several annotations of which step we are. Therefore, we collect them all and then 
                    # we will select the last one as it is the last achieved step. 

        for line in error_log_3_lines:
            
            if 'remaining wall clock time:     0 s ' in line: # If there exists this string in some line, then it means that before this it will have the steps. 
                
                steps.append(line) #We add it to the steps list. 
                
               
            
        if len(steps) != 0: # If the steps list is not empty, as maybe there are error3.log that do not have this line because it fails before time
            
            values_dict_100ns.append(steps[-1]) # Then we add the last value of steps to the values that we want to add to the dictionary. 
            PDBS_finished_completed.append(i[0:4])
            
        
        
        
        if len(steps) == 0: # If the steps list is empty, this means that we do not have the number of steps, meaning that there maybe an error. 
            if "will finish" in error_log_3_lines[-1]:
                
                splitted_string = str(error_log_3_lines[-1]).split(" ")
                num = int(splitted_string[1][0:len(splitted_string[1])-1])
                perc = round((num/50000000)*100)
                values_dict_100ns.append('Not steps indication // Not finished, percentage finished = '+str(perc)+'%') # We indicate this and also we print the possible error that lays in the last line of the error3.log
                
                
            else:
                values_dict_100ns.append('Not steps indication // Error in previous sbatches: "'+str(error_log_3_lines[-1])+'"') # We indicate this and also we print the possible error that lays in the last line of the error3.log
            
                if i[0:4] not in PDBs_with_errors:
                    PDBs_with_errors.append(i[0:4])
                
        
        
        
        dictio_100ns[i[0:4]] = values_dict_100ns # We add again (and rewrite if previous written) all this info into the dictionary
                                                 # of those that have 100ns folder. 
    
    # For knowing if they are using Gromos forcefield or Charm forcefield for the lipid membrane: 
    
        
    
    with open('/home/imartinv/Escritorio/ventana_a_starlife/CNS_PD_Memprot_dynamics/'+i+'/topol.top') as topol_top:
            
        topol_top_lines_f = topol_top.readlines() # We read line by line the error3.log file
    
    for line_top in topol_top_lines_f:
        
        if 'gromos53a6' in line_top and 'charmm36' in line_top:
            both_forcefields_pdb.append(i[0:4])
            both = both+1
            break
    
        elif 'gromos53a6'in line_top:
            gromos_forcefields_pdb.append(i[0:4])
            gromos_forcefield_num = gromos_forcefield_num+1
            break

        elif 'charmm36' in line_top:
            charmm_forcefields_pdb.append(i[0:4])
            charm_forcefield_num = charm_forcefield_num+1
            break



In [19]:
#print('PDB that had already run or are running: '+str(len(dictio_100ns)))

#print('· From this, PDBs with error: '+str(len(PDBs_with_errors)))

#print('· From this, PDBs that had already run and correct: '+str(len(PDBS_finished_completed)))

#print('· From this, PDBs that are running: '+str(len(dictio_100ns)-(len(PDBs_with_errors)+len(PDBS_finished_completed))))

#print('PDBs that had not run: ', str(len(dictio_not_100ns_notrunned)))

PDB that had already run or are running: 59
· From this, PDBs with error: 8
· From this, PDBs that had already run and correct: 22
· From this, PDBs that are running: 29
PDBs that had not run:  75


In [4]:
#print('PDB that had already run or are running: '+str(len(dictio_100ns)))

#print('· From this, PDBs with error: '+str(len(PDBs_with_errors)))

#print('· From this, PDBs that had already run and correct: '+str(len(PDBS_finished_completed)))

#print('· From this, PDBs that are running: '+str(len(dictio_100ns)-(len(PDBs_with_errors)+len(PDBS_finished_completed))))

#print('PDBs that had not run: ', str(len(dictio_not_100ns_notrunned)))

PDB that had already run or are running: 67
· From this, PDBs with error: 8
· From this, PDBs that had already run and correct: 42
· From this, PDBs that are running: 17
PDBs that had not run:  67


In [9]:
#print('PDB that had already run or are running: '+str(len(dictio_100ns)))

#print('· From this, PDBs with error: '+str(len(PDBs_with_errors)))

#print('· From this, PDBs that had already run and correct: '+str(len(PDBS_finished_completed)))

#print('· From this, PDBs that are running: '+str(len(dictio_100ns)-(len(PDBs_with_errors)+len(PDBS_finished_completed))))

#print('PDBs that had not run: ', str(len(dictio_not_100ns_notrunned)))

PDB that had already run or are running: 67
· From this, PDBs with error: 8
· From this, PDBs that had already run and correct: 46
· From this, PDBs that are running: 13
PDBs that had not run:  67


In [4]:
#print('PDB that had already run or are running: '+str(len(dictio_100ns)))

#print('· From this, PDBs with error: '+str(len(PDBs_with_errors)))

#print('· From this, PDBs that had already run and correct: '+str(len(PDBS_finished_completed)))

#print('· From this, PDBs that are running: '+str(len(dictio_100ns)-(len(PDBs_with_errors)+len(PDBS_finished_completed))))

#print('PDBs that had not run: ', str(len(dictio_not_100ns_notrunned)))

PDB that had already run or are running: 83
· From this, PDBs with error: 10
· From this, PDBs that had already run and correct: 52
· From this, PDBs that are running: 21
PDBs that had not run:  51


In [10]:
#print('PDB that had already run or are running: '+str(len(dictio_100ns)))

#print('· From this, PDBs with error: '+str(len(PDBs_with_errors)))

#print('· From this, PDBs that had already run and correct: '+str(len(PDBS_finished_completed)))

#print('· From this, PDBs that are running: '+str(len(dictio_100ns)-(len(PDBs_with_errors)+len(PDBS_finished_completed))))

#print('PDBs that had not run: ', str(len(dictio_not_100ns_notrunned)))

PDB that had already run or are running: 112
· From this, PDBs with error: 16
· From this, PDBs that had already run and correct: 52
· From this, PDBs that are running: 44
PDBs that had not run:  22


In [4]:
print('PDB that had already run or are running: '+str(len(dictio_100ns)))

print('· From this, PDBs with error: '+str(len(PDBs_with_errors)))

print('· From this, PDBs that had already run and correct: '+str(len(PDBS_finished_completed)))

print('· From this, PDBs that are running: '+str(len(dictio_100ns)-(len(PDBs_with_errors)+len(PDBS_finished_completed))))

print('PDBs that had not run: ', str(len(dictio_not_100ns_notrunned)))

PDB that had already run or are running: 63
· From this, PDBs with error: 16
· From this, PDBs that had already run and correct: 3
· From this, PDBs that are running: 44
PDBs that had not run:  22


In [6]:
print('PDB that had already run or are running: '+str(len(dictio_100ns)))

print('· From this, PDBs with error: '+str(len(PDBs_with_errors)))

print('· From this, PDBs that had already run and correct: '+str(len(PDBS_finished_completed)))

print('· From this, PDBs that are running: '+str(len(dictio_100ns)-(len(PDBs_with_errors)+len(PDBS_finished_completed))))

print('PDBs that had not run: ', str(len(dictio_not_100ns_notrunned)))

PDB that had already run or are running: 77
· From this, PDBs with error: 72
· From this, PDBs that had already run and correct: 4
· From this, PDBs that are running: 1
PDBs that had not run:  17


In [35]:
print('PDBs that had already run or are running: \n')
dictio_100ns

PDBs that had already run or are running: 



{'5iu7': ['100ns --> full',
  'step 50000000, remaining wall clock time:     0 s          '],
 '2lm2': ['100ns --> empty // Error in previous sbatches',
  'Not steps indication // Error in previous sbatches: "system msg for write_line failure : Bad file descriptor"'],
 '5olo': ['100ns --> full',
  'step 50000000, remaining wall clock time:     0 s          '],
 '6d1s': ['100ns --> full',
  'Not steps indication // Not finished, percentage finished = 88%'],
 '2ksr': ['100ns --> full',
  'step 50000000, remaining wall clock time:     0 s          '],
 '6jzh': ['100ns --> full',
  'Not steps indication // Not finished, percentage finished = 75%'],
 '5wf6': ['100ns --> full',
  'step 50000000, remaining wall clock time:     0 s          '],
 '2k58': ['100ns --> full',
  'step 50000000, remaining wall clock time:     0 s          '],
 '5uvi': ['100ns --> full',
  'step 50000000, remaining wall clock time:     0 s          '],
 '3qak': ['100ns --> full',
  'step 50000000, remaining wall cloc

In [11]:
print('PDBs with errors: \n')

PDBs_with_errors.sort()
PDBs_with_errors

PDBs with errors: 



['2lm2',
 '3pwh',
 '5ek0',
 '5g53',
 '5kxi',
 '5o8f',
 '5olh',
 '5om4',
 '6d6u',
 '6gdg',
 '6kr8',
 '6me5',
 '6n4r',
 '6ni3',
 '6oik',
 '6wgt']

In [10]:
print('PDBs that had already run and correct \n')
PDBS_finished_completed.sort()
PDBS_finished_completed

PDBs that had already run and correct 



['2k58',
 '2ksr',
 '2r4s',
 '2ydo',
 '3eml',
 '3kj6',
 '3nya',
 '3p0g',
 '3pbl',
 '3pds',
 '3qak',
 '3rze',
 '3vg9',
 '3vga',
 '4cof',
 '4eiy',
 '4gbr',
 '4mqs',
 '4mqt',
 '4ug2',
 '4uhr',
 '5cxv',
 '5d5b',
 '5iu7',
 '5iu8',
 '5iua',
 '5jqh',
 '5jtb',
 '5mzp',
 '5n2r',
 '5nlx',
 '5ojm',
 '5olo',
 '5om1',
 '5uig',
 '5uvi',
 '5vra',
 '5wf5',
 '5wf6',
 '5wiu',
 '5wiv',
 '5x7d',
 '5yc8',
 '5zk3',
 '5zk8',
 '5zkb',
 '6a93',
 '6a94',
 '6aqf',
 '6bqg',
 '6bqh',
 '6cm4']

In [11]:
print('PDBs that had not run: \n')
dictio_not_100ns_notrunned.sort()
dictio_not_100ns_notrunned

PDBs that had not run: 



['6kr8',
 '6kuw',
 '6kux',
 '6kuy',
 '6me2',
 '6me3',
 '6me5',
 '6me8',
 '6n4q',
 '6n4r',
 '6ni3',
 '6nt3',
 '6nt4',
 '6oij',
 '6oik',
 '6ps0',
 '6ps2',
 '6ps5',
 '6ps7',
 '6ps8',
 '6qfa',
 '6ur8',
 '6usf',
 '6vms',
 '6vxo',
 '6w6o',
 '6wgt',
 '6wh4',
 '6wha',
 '6wjc',
 '6x3u',
 '6zdr',
 '6zdv',
 '7bz2',
 '7cmu',
 '7dfp',
 '7dhi',
 '7dhr',
 '7dtc',
 '7dtd',
 '7e2y',
 '7e2z',
 '7eoq',
 '7eor',
 '7eos',
 '7eot',
 '7eou',
 '7jvr',
 '7k48',
 '7mix',
 '7miy']

In [18]:
print('Number of PDBs that use GROMOS Forcefield:',gromos_forcefield_num)

Number of PDBs that use GROMOS Forcefield: 59


In [19]:
print('Number of PDBs that use CHARM36 Forcefield:',charm_forcefield_num)

Number of PDBs that use CHARM36 Forcefield: 98


In [36]:
print('Number of PDBs that use both Forcefields (if that is possible): ',both)

Number of PDBs that use both Forcefields (if that is possible):  0


In [37]:
gromos_forcefields_pdb.sort()
gromos_forcefields_pdb

['2k58',
 '2ksr',
 '2lm2',
 '2r4s',
 '2ydo',
 '3eml',
 '3kj6',
 '3nya',
 '3p0g',
 '3pbl',
 '3pds',
 '3pwh',
 '3qak',
 '3rze',
 '3sn6',
 '3vg9',
 '3vga',
 '4cof',
 '4eiy',
 '4gbr',
 '4mqs',
 '4mqt',
 '4ug2',
 '4uhr',
 '5cxv',
 '5d5b',
 '5ek0',
 '5g53',
 '5iu7',
 '5iu8',
 '5iua',
 '5jqh',
 '5jtb',
 '5kxi',
 '5mzp',
 '5n2r',
 '5nlx',
 '5o8f',
 '5ojm',
 '5olh',
 '5olo',
 '5om1',
 '5om4',
 '5uig',
 '5uvi',
 '5vra',
 '5wf5',
 '5wf6',
 '5wiu',
 '5wiv',
 '5x7d',
 '6aqf',
 '6bqg',
 '6bqh']

In [38]:
charmm_forcefields_pdb.sort()
charmm_forcefields_pdb

['5yc8',
 '5zk3',
 '5zk8',
 '5zkb',
 '6a93',
 '6a94',
 '6a96',
 '6cm4',
 '6cnj',
 '6cnk',
 '6d1s',
 '6d6t',
 '6d6u',
 '6e67',
 '6gdg',
 '6gt3',
 '6hug',
 '6huj',
 '6huo',
 '6hup',
 '6i53',
 '6ira',
 '6irg',
 '6irh',
 '6j8e',
 '6j8h',
 '6jzh',
 '6k41',
 '6k42',
 '6kr8',
 '6kuw',
 '6kux',
 '6kuy',
 '6me2',
 '6me3',
 '6me5',
 '6me8',
 '6n4q',
 '6n4r',
 '6ni3',
 '6nt3',
 '6nt4',
 '6oij',
 '6oik',
 '6ps0',
 '6ps2',
 '6ps5',
 '6ps7',
 '6ps8',
 '6qfa',
 '6ur8',
 '6usf',
 '6vms',
 '6vxo',
 '6w6o',
 '6wgt',
 '6wh4',
 '6wha',
 '6wjc',
 '6x3u',
 '6zdr',
 '6zdv',
 '7bz2',
 '7cmu',
 '7dfp',
 '7dhi',
 '7dhr',
 '7dtc',
 '7dtd',
 '7e2y',
 '7e2z',
 '7eoq',
 '7eor',
 '7eos',
 '7eot',
 '7eou',
 '7jvr',
 '7k48',
 '7mix',
 '7miy']

In [29]:
#k = 4

#with open('PDBS_finished_completed_'+str(k)+'.txt', 'w') as finished_file:
    for listitem in PDBS_finished_completed:
        finished_file.write(f'{listitem}\n')


mwf -top md.tpr -traj md.xtc -char md.tpr -pr 4 -filt -i topology --trust cohbonds



In [31]:
finished_run_pdbs_ANTERIOR = []

with open('/home/imartinv/Escritorio/TFG/1.LIST_PROTEINS/Isa_dinámicas_preparacion/PDBS_finished_completed_'+str(k-1)+'.txt', 'r') as finished_file:
    for line in finished_file:
        # Remove linebreak which is the last character of the string
        curr_place = line[:-1]
        # Add item to the list
        finished_run_pdbs_ANTERIOR.append(curr_place)


In [32]:
finished_run_pdbs_ACTUAL = []

with open('/home/imartinv/Escritorio/TFG/1.LIST_PROTEINS/Isa_dinámicas_preparacion/PDBS_finished_completed_'+str(k)+'.txt', 'r') as finished_file:
    for line in finished_file:
        # Remove linebreak which is the last character of the string
        curr_place = line[:-1]
        # Add item to the list
        finished_run_pdbs_ACTUAL.append(curr_place)

In [25]:
finished_run_pdbs_difference = []

for i in finished_run_pdbs_ACTUAL: 
    if i not in finished_run_pdbs_ANTERIOR:
        finished_run_pdbs_difference.append(i)
        

In [27]:
finished_run_pdbs_difference

['4uhr', '5ojm', '6bqg', '6bqh']

In [33]:
with open('PDBS_finished_difference.txt', 'w') as finished_file:
    for listitem in finished_run_pdbs_difference_2:
        finished_file.write(f'{listitem}\n')


In [6]:
with open('PDBS_finished_completed_toremovextc.txt', 'w') as finished_file:
    for listitem in PDBS_finished_completed:
        finished_file.write(f'{listitem}\n')


### BSC had a power outage. 
I am going to see which jobs where in the second one and which in the third one. If they are in the third one, we can run the sbatch_cont but if failed in the second (or first) we have to create new ones. 

In [31]:
# We pay attention to this folder because it is the one resultant of the simulation and it is the one that 
# contains the trajectory of the protein. 

pdbs_stoppedrun_thirdjob = []

pdbs_stoppedrun_secondjob = []


for i in PDBs_name_starlife: # For each PDB folder in starlife


    
    files = os.listdir('/home/imartinv/Escritorio/ventana_a_starlife/CNS_PD_Memprot_dynamics/'+i) # We list the items inside each PDB folder

    # See which have the error3.log (which means they had already run or were running)
    if 'error3.log' in files: 
        
        error_log_3_lines = []
        
        with open('/home/imartinv/Escritorio/ventana_a_starlife/CNS_PD_Memprot_dynamics/'+i+'/error3.log') as error_log_3:
            
            error_log_3_lines_f = error_log_3.readlines() # We read line by line the error3.log file
            
        for sub in error_log_3_lines_f:
            
            error_log_3_lines.append(sub.replace("\n", "")) # To eliminate the '\n' character each time we read a line.We substitute for nothing.
        
        
        if '/bin/bash: /apps/modules/LMOD/7.7/lmod/lmod/init/bash: Error de entrada/salida' in error_log_3_lines: 
            
            pdbs_stoppedrun_thirdjob.append(i)
            
        
    # See which have the error3.log (which means they had already run or were running)
    if 'error2.log' in files: 
        
        error_log_2_lines = []
        
        with open('/home/imartinv/Escritorio/ventana_a_starlife/CNS_PD_Memprot_dynamics/'+i+'/error2.log') as error_log_2:
            
            error_log_2_lines_f = error_log_2.readlines() # We read line by line the error3.log file
            
        for sub in error_log_2_lines_f:
            
            error_log_2_lines.append(sub.replace("\n", "")) # To eliminate the '\n' character each time we read a line.We substitute for nothing.
        
        
        if '/bin/bash: /apps/modules/LMOD/7.7/lmod/lmod/init/bash: Error de entrada/salida' in error_log_2_lines: 
            
            pdbs_stoppedrun_secondjob.append(i[0:4])
            
    
    

        
    
    
    
    

In [29]:
pdbs_stoppedrun_secondjob

[]

In [33]:
pdbs_stoppedrun_thirdjob

['6d1s_default_dppc-atomistic-simulation',
 '6jzh_default_dppc-atomistic-simulation',
 '7dhr_default_dppc-atomistic-simulation',
 '6a96_default_dppc-atomistic-simulation',
 '6j8h_default_dppc-atomistic-simulation',
 '6irh_default_dppc-atomistic-simulation',
 '6zdr_default_dppc-atomistic-simulation',
 '6ps2_default_dppc-atomistic-simulation',
 '6wjc_default_dppc-atomistic-simulation',
 '6ps0_default_dppc-atomistic-simulation',
 '6e67_default_dppc-atomistic-simulation',
 '6wgt_default_dppc-atomistic-simulation',
 '6ps8_default_dppc-atomistic-simulation',
 '6vms_default_dppc-atomistic-simulation',
 '6kuw_default_dppc-atomistic-simulation',
 '7eoq_default_dppc-atomistic-simulation',
 '6qfa_default_dppc-atomistic-simulation',
 '7e2z_default_dppc-atomistic-simulation',
 '6i53_default_dppc-atomistic-simulation',
 '6gt3_default_dppc-atomistic-simulation',
 '6hug_default_dppc-atomistic-simulation',
 '6d6t_default_dppc-atomistic-simulation',
 '7dtd_default_dppc-atomistic-simulation',
 '6k42_defa

In [34]:
pdbs_error_or_notrun = []

for pdb in PDBs_name_starlife: 
    if pdb not in pdbs_stoppedrun_thirdjob: 
        pdbs_error_or_notrun.append(pdb)
        

In [36]:
pdbs_error_or_notrun.sort()
pdbs_error_or_notrun

['2lm2_default_dppc-atomistic-simulation',
 '3pwh_default_dppc-atomistic-simulation',
 '3sn6_default_dppc-atomistic-simulation',
 '4cof_default_dppc-atomistic-simulation',
 '5ek0_default_dppc-atomistic-simulation',
 '5g53_default_dppc-atomistic-simulation',
 '5iu7_default_dppc-atomistic-simulation',
 '5kxi_default_dppc-atomistic-simulation',
 '5o8f_default_dppc-atomistic-simulation',
 '5ojm_default_dppc-atomistic-simulation',
 '5olh_default_dppc-atomistic-simulation',
 '5om4_default_dppc-atomistic-simulation',
 '6d6u_default_dppc-atomistic-simulation',
 '6gdg_default_dppc-atomistic-simulation',
 '6j8e_default_dppc-atomistic-simulation',
 '6kr8_default_dppc-atomistic-simulation',
 '6me5_default_dppc-atomistic-simulation',
 '6n4r_default_dppc-atomistic-simulation',
 '6ni3_default_dppc-atomistic-simulation',
 '7eor_default_dppc-atomistic-simulation',
 '7eos_default_dppc-atomistic-simulation',
 '7eot_default_dppc-atomistic-simulation',
 '7eou_default_dppc-atomistic-simulation',
 '7jvr_defa

In [ ]:
['2lm2',
 '3pwh',
 '5ek0',
 '5g53',
 '5kxi',
 '5o8f',
 '5olh',
 '5om4',
 '6d6u',
 '6gdg',
 '6kr8',
 '6me5',
 '6n4r',
 '6ni3',
 '6oik',
 '6wgt']

In [37]:
dictio_not_100ns_notrunned

['7dhr',
 '7mix',
 '7e2z',
 '7dtd',
 '7eos',
 '7e2y',
 '7bz2',
 '7jvr',
 '7dhi',
 '7miy',
 '7cmu',
 '7eou',
 '7eor',
 '7k48',
 '7eot',
 '7dfp',
 '7dtc']

In [39]:
textfile = open("pdbs_stoppedrun_thirdjob.txt", "w")

for element in pdbs_stoppedrun_thirdjob:

    textfile.write(element + "\n")

textfile.close()